# Import

In [1]:
# settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [62]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network
from IPython.display import HTML, display
from gensim.models.poincare import PoincareModel, PoincareRelations
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from ipywidgets import widgets, interact, interactive

from src.load_datasets import load_dataset, load_rankings
from src.encoding import load_graph

In [3]:
# figure settings
sns.set()
sns.set(style='whitegrid', )
#sns.set_palette(palette="Set2")
sns.set_palette(palette="Paired")

# set seed
np.random.seed(7)

# Get data

In [4]:
dataset = load_dataset('../../data/raw/dataset.csv')
rankings = load_rankings('../../data/raw/rankings.csv')

Loading data ...
Loading rankings ...


# Get graph

In [5]:
# get graph as networkx graph
G = load_graph("../../data/raw/graph.adjlist")
# show some information about the graph
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print("Graph information:")
print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")

Number of nodes: 45
Number of edges: 44


### Visualize the graph

In [6]:
# create a Pyvis Network instance
nt = Network(heading="Hierarchical Structure Feature Encoder", notebook=True, cdn_resources='in_line')
# add nodes and edges to the Pyvis Network
nt.from_nx(G)
# show the Pyvis Network
html = nt.generate_html('encoder_graph.html')
display(HTML(html))

### Vector representations of those no de labels plotted on a 2d scatter graph
For a better visualization and interpretation I am adding the level feature as indicator if the nodes were actually in the same level of the hierarchy. The level feature is calculated by the degree of the node. The higher the degree the higher the level of the node. The level feature is then added as a color and size feature to the scatter plot. Another option would be to use the degree as a size feature and the level as a color feature.

In [7]:
def calculate_level(G):
    """
    Generates the feature hierarchical level for each node in the graph. Where the level defines the depth of the node in the graph hierarchy.

    :param G: The input graph.
    :type G: networkx.Graph

    :return: The graph with the hierarchical level.
    """
    levels = {}  # Dictionary to store hierarchical levels of nodes

    # Iterate over all nodes in the graph and calculate their hierarchical levels
    for node in G.nodes:
        if G.degree(node) == 0:
            levels[node] = 0
        else:
            max_level = 0
            for neighbor in G.neighbors(node):
                if neighbor in levels:
                    max_level = max(max_level, levels[neighbor] + 1)
            levels[node] = max_level

    # Add the hierarchical level as an attribute to each node in the graph
    nx.set_node_attributes(G, levels, "hierarchical_level")

    return G

In [8]:
# Get hierarchical level for each node
G = calculate_level(G)

In [9]:
# Print the hierarchical level for all nodes and make mapping dict
level_dict = {}
for node in G.nodes:
    print(f"Node: {node}, Level: {G.nodes[node]['hierarchical_level']}")
    level_dict[node] = G.nodes[node]['hierarchical_level']

Node: BE, Level: 0
Node: identifier, Level: 1
Node: BUCV10RGLMME, Level: 0
Node: bucvglmm, Level: 1
Node: BUCV10TE, Level: 0
Node: bucvte, Level: 1
Node: BUCV2RGLMME, Level: 2
Node: BUCV2TE, Level: 2
Node: BUCV5RGLMME, Level: 2
Node: BUCV5TE, Level: 2
Node: CBE, Level: 0
Node: supervised, Level: 1
Node: CE, Level: 0
Node: unsupervised, Level: 2
Node: CV10RGLMME, Level: 0
Node: cvglmm, Level: 1
Node: CV10TE, Level: 0
Node: cvte, Level: 1
Node: CV2RGLMME, Level: 2
Node: CV2TE, Level: 2
Node: CV5RGLMME, Level: 2
Node: CV5TE, Level: 2
Node: DE, Level: 3
Node: DTEM10, Level: 0
Node: dte, Level: 2
Node: DTEM2, Level: 3
Node: DTEM5, Level: 3
Node: ME01E, Level: 0
Node: mee, Level: 2
Node: ME10E, Level: 3
Node: ME1E, Level: 3
Node: MHE, Level: 3
Node: OE, Level: 2
Node: OHE, Level: 2
Node: PBTE0001, Level: 0
Node: pbte, Level: 2
Node: PBTE001, Level: 3
Node: PBTE01, Level: 3
Node: RGLMME, Level: 0
Node: glmm, Level: 2
Node: SE, Level: 3
Node: TE, Level: 0
Node: te, Level: 2
Node: WOEE, Level: 

In [100]:
# Print the degree for all nodes and make mapping dict
degree_dict = {}
for node in G.nodes:
    print(f"Node: {node}, Degree: {G.degree(node)}")
    degree_dict[node] = G.degree(node)

Node: BE, Degree: 1
Node: identifier, Degree: 4
Node: BUCV10RGLMME, Degree: 1
Node: bucvglmm, Degree: 4
Node: BUCV10TE, Degree: 1
Node: bucvte, Degree: 4
Node: BUCV2RGLMME, Degree: 1
Node: BUCV2TE, Degree: 1
Node: BUCV5RGLMME, Degree: 1
Node: BUCV5TE, Degree: 1
Node: CBE, Degree: 1
Node: supervised, Degree: 8
Node: CE, Degree: 1
Node: unsupervised, Degree: 6
Node: CV10RGLMME, Degree: 1
Node: cvglmm, Degree: 4
Node: CV10TE, Degree: 1
Node: cvte, Degree: 4
Node: CV2RGLMME, Degree: 1
Node: CV2TE, Degree: 1
Node: CV5RGLMME, Degree: 1
Node: CV5TE, Degree: 1
Node: DE, Degree: 1
Node: DTEM10, Degree: 1
Node: dte, Degree: 4
Node: DTEM2, Degree: 1
Node: DTEM5, Degree: 1
Node: ME01E, Degree: 1
Node: mee, Degree: 4
Node: ME10E, Degree: 1
Node: ME1E, Degree: 1
Node: MHE, Degree: 1
Node: OE, Degree: 1
Node: OHE, Degree: 1
Node: PBTE0001, Degree: 1
Node: pbte, Degree: 4
Node: PBTE001, Degree: 1
Node: PBTE01, Degree: 1
Node: RGLMME, Degree: 1
Node: glmm, Degree: 4
Node: SE, Degree: 1
Node: TE, Degree

In [10]:
# Get the position of each node in the graph
pos = nx.spring_layout(G, k=0.5, iterations=50)

In [102]:
# Extract x and y coordinates and combine with the level attribute
graph_df = pd.DataFrame.from_dict(pos, orient='index', columns=['x', 'y'])
graph_df['level'] = nx.get_node_attributes(G, 'hierarchical_level')
graph_df['degree'] = pd.Series(degree_dict)

In [104]:
graph_df

x         y  level  degree
BE           -0.639424  0.616036      0       1
identifier   -0.229179  0.352920      1       4
BUCV10RGLMME  0.308321 -0.587731      0       1
bucvglmm     -0.190442 -0.771191      1       4
BUCV10TE     -0.408692  0.711284      0       1
bucvte       -0.029276  0.496480      1       4
BUCV2RGLMME  -0.425016 -1.000000      2       1
BUCV2TE      -0.302639  0.868658      2       1
BUCV5RGLMME  -0.529198 -0.807328      2       1
BUCV5TE       0.394581 -0.035033      2       1
CBE           0.481950  0.623366      0       1
supervised    0.184943  0.220673      1       8
CE            0.254986 -0.088264      0       1
unsupervised -0.362252  0.069257      2       6
CV10RGLMME    0.264059 -0.775264      0       1
cvglmm        0.003991 -0.723041      1       4
CV10TE       -0.348937 -0.223734      0       1
cvte          0.050406 -0.112240      1       4
CV2RGLMME     0.334399 -0.907622      2       1
CV2TE         0.073075 -0.527375      2       1
CV5RGLMME    -0.181258 -0.956800      2       1
CV5TE         0.526379 -0.067309      2       1
DE           -0.512618  0.246409      3       1
DTEM10       -0.279921  0.958678      0       1
dte          -0.237761  0.575953      2       4
DTEM2        -0.591894  0.332660      3       1
DTEM5        -0.077566  0.712771      3       1
ME01E        -0.059308 -0.270140      0       1
mee           0.334988  0.096914      2       4
ME10E         0.506694  0.396549      3       1
ME1E          0.794319  0.225299      3       1
MHE          -0.800746 -0.059605      3       1
OE            0.074996 -0.054844      2       1
OHE           0.142165  0.623998      2       1
PBTE0001      0.853806 -0.472197      0       1
pbte          0.561077 -0.248333      2       4
PBTE001       0.246699 -0.536790      3       1
PBTE01        0.913381 -0.196441      3       1
RGLMME       -0.589580 -0.586340      0       1
glmm         -0.174896 -0.399207      2       4
SE           -0.832364  0.117996      3       1
TE            0.065376  0.884551      0       1
te            0.113395  0.418523      2       4
WOEE          0.320832  0.712278      2       1
encoder      -0.001848  0.145576      3       2

In [135]:
# Make scatter plot of the nodes
plt.figure(figsize=(10, 8))
sns.scatterplot(data=graph_df, x='x', y='y', size='degree', sizes=(80, 400), hue='level', palette='Set2')
# Add node labels as annotations to the plot
for node, (x, y) in pos.items():
    plt.text(x, y, node, fontsize=10)
# Add title to the plot
plt.title('Graph Nodes Scatterplot', fontweight='bold')
# Add legend
plt.legend(labelspacing=1, borderpad=1, loc='upper right', bbox_to_anchor=(1.2, 1))
# Show plot
plt.tight_layout()
plt.show()

# Poincarè embedding
We want to encode the feature encoder using the [Poincarè Embedding](https://radimrehurek.com/gensim/models/poincare.html). By doing this we encode the hierarchical structure of the feature encoder and preserve it. The graph with the hierarchical structure is given in [data/raw](../../data/raw), the library networkx is used to load the graph.

Hierarchies are often trees where the number of nodes increases exponentially by the level, so we want a space that increases exponentially as we deviate from the “origin” of the space. The Poincaré ball follows this property, so we can define our vectors on this space. We define a loss function that gives a good score for clustering vectors in the same hierarchy and spreading out vectors in different hierarchies. Finally, we can find vectors to minimize our loss function using Riemann gradient descent and get a good numerical representation of the words being the name of the encoders.

The method is described in [Maximilian Nickel, Douwe Kiela - “Poincaré Embeddings for Learning Hierarchical Representations”](https://arxiv.org/abs/1705.08039)

## Run Embedding

In [13]:
%%time
# Extract all relations form the graph
relations = []
for edge in G.edges():
    relations.append((edge[0], edge[1]))

CPU times: total: 0 ns
Wall time: 0 ns


In [64]:
%%time
# Embed the graph
model = PoincareModel(relations, seed=7)
# Train the model
model.train(epochs=5000, print_every=500)

CPU times: total: 2.19 s
Wall time: 21.7 s


In [65]:
# Get the embeddings (vector representation) of the nodes hierarchy structure
embeddings = model.kv.vectors
# Map embeddings (vector representation) to node names
embeddings_dict = {}
for node in G.nodes:
    embeddings_dict[node] = model.kv[node]

In [198]:
# Transform embeddings (vector representation) to a dataframe
embeddings_df = pd.DataFrame(embeddings)
# Add node names as a column to the dataframe
embeddings_df['node'] = embeddings_dict.keys()
embeddings_df = embeddings_df.set_index('node')
# Rename columns to dimension_1, dimension_2, ...
embeddings_df.columns = [f'dimension_{i}' for i in range(1, embeddings_df.shape[1] + 1)]
embeddings_df

dimension_1  dimension_2  dimension_3  dimension_4  dimension_5  \
node                                                                            
BE               0.017022     0.093297     0.177198    -0.010911    -0.021327   
identifier       0.016701     0.090865     0.172922    -0.010525    -0.020648   
BUCV10RGLMME     0.016611     0.090991     0.172834    -0.010636    -0.020788   
bucvglmm         0.016621     0.090839     0.172649    -0.010588    -0.020717   
BUCV10TE         0.020424     0.061432     0.144713     0.001309    -0.003877   
bucvte           0.232103    -0.063685    -0.131307    -0.117467    -0.124104   
BUCV2RGLMME      0.208804    -0.056545    -0.119364    -0.106077    -0.112729   
BUCV2TE          0.207658    -0.056234    -0.118692    -0.105492    -0.112102   
BUCV5RGLMME      0.206398    -0.055730    -0.118251    -0.104931    -0.111655   
BUCV5TE          0.046305    -0.003784    -0.065519    -0.029400    -0.047344   
CBE              0.032686    -0.004198    -0.115525     0.029374     0.095290   
supervised       0.028475    -0.003150    -0.103101     0.026032     0.085064   
CE               0.026717    -0.002658    -0.101029     0.025352     0.083706   
unsupervised     0.027658    -0.002965    -0.101573     0.025618     0.083980   
CV10RGLMME      -0.042318     0.013816    -0.024729     0.001099     0.026391   
cvglmm          -0.056206    -0.092243    -0.047945     0.059887     0.078368   
CV10TE          -0.020306    -0.030054    -0.004445     0.022072     0.025807   
cvte            -0.017923    -0.030583    -0.002796     0.021630     0.024140   
CV2RGLMME        0.000144    -0.094118    -0.017576     0.017380     0.039700   
CV2TE            0.000099     0.009849     0.046180     0.007858     0.006479   
CV5RGLMME       -0.010218    -0.000575     0.090650     0.027844    -0.051827   
CV5TE           -0.025029     0.004274     0.038286     0.033750     0.064254   
DE               0.032564     0.061434     0.181504     0.012355     0.009099   
DTEM10           0.017961     0.062054     0.138141    -0.001075    -0.006761   
dte              0.019634     0.060530     0.141283     0.000907    -0.004271   
DTEM2            0.019586     0.061157     0.141923     0.000671    -0.004604   
DTEM5            0.025860     0.088603    -0.144764    -0.056261    -0.140115   
ME01E            0.024102     0.074977    -0.126156    -0.049230    -0.120904   
mee              0.024033     0.076779    -0.126632    -0.049734    -0.122785   
ME10E            0.025525     0.075342    -0.126030    -0.049949    -0.122070   
ME1E            -0.203868     0.123075     0.004735    -0.061633    -0.015492   
MHE             -0.180126     0.108918     0.003524    -0.054469    -0.013299   
OE              -0.179032     0.108245     0.003054    -0.054081    -0.013014   
OHE             -0.180855     0.109365     0.003452    -0.054706    -0.013378   
PBTE0001         0.002105    -0.136081    -0.025751     0.023185     0.055390   
pbte             0.000674    -0.090734    -0.016350     0.016462     0.037953   
PBTE001          0.000454    -0.094093    -0.017245     0.017387     0.039393   
PBTE01          -0.013570     0.002058     0.135755     0.039765    -0.079296   
RGLMME          -0.009726    -0.000557     0.086251     0.026455    -0.049295   
glmm            -0.009192    -0.000364     0.090625     0.027176    -0.051868   
SE              -0.037684     0.009465     0.058453     0.050212     0.097877   
TE              -0.024376     0.003493     0.039025     0.035109     0.067138   
te              -0.025255     0.004692     0.039726     0.034679     0.066245   
WOEE             0.118508    -0.019767    -0.191826    -0.075681    -0.115987   
encoder         -0.133533     0.032725    -0.095313     0.008496     0.090677   

              dimension_6  dimension_7  dimension_8  dimension_9  \
node                                                               
BE              -0.107233    -0.092924     0.065719     0.127601   
id

## Build Encoder function

In [212]:
def poincare_encoding(path_to_graph, encode_dim=None, epochs=10000, seed=7):
    """
    Generates the Poincarè embedding for the given graph and encodes the given column with it.

    :param encode_dim:
    :param path_to_graph: The path to the graph.
    :type path_to_graph: str
    :param data: The data with the column to encode.
    :type data: pandas.DataFrame
    :param encode_column: The column to encode.
    :type encode_column: str
    :param enncode_dim: The dimension of the embedding.
    :type enncode_dim: int
    :param epochs: The number of epochs to train the embedding.
    :type epochs: int
    :param seed: The seed for the embedding.
    :type seed: int

    :return: The encoded data.
    :rtype: pandas.DataFrame
    """
    # load Graph
    G = load_graph(path_to_graph)
    # Extract all relations form the graph
    relations = []
    for edge in G.edges():
        relations.append((edge[0], edge[1]))
    # Embed the graph
    if encode_dim is None:
        model = PoincareModel(relations, seed=seed)
    else:
        model = PoincareModel(relations, seed=seed, size=encode_dim)
    # Train the model
    model.train(epochs=epochs, print_every=500)
    # Get the embeddings and map them to the node names
    embeddings_dict = {}
    for node in G.nodes:
        embeddings_dict[node] = model.kv[node]
    # Transform embeddings to a dataframe
    embeddings_df = pd.DataFrame(embeddings_dict).T
    # Rename columns to dimension_1, dimension_2, ...
    embeddings_df.columns = [f'dimension_{i}' for i in range(1, embeddings_df.shape[1] + 1)]

    return embeddings_df

In [215]:
%%time
df = poincare_encoding("../../data/raw/graph.adjlist")
df

CPU times: total: 15 s
Wall time: 42 s


dimension_1  dimension_2  dimension_3  dimension_4  dimension_5  \
BE               0.016805     0.093824     0.177279    -0.011248    -0.021775   
identifier       0.016386     0.091459     0.172815    -0.010962    -0.021217   
BUCV10RGLMME     0.229952    -0.058453    -0.134558    -0.118702    -0.128329   
bucvglmm         0.199785    -0.050071    -0.117406    -0.103469    -0.112219   
BUCV10TE         0.026064     0.001051    -0.112886     0.026197     0.092278   
bucvte           0.021630     0.001586    -0.099934     0.022788     0.081579   
BUCV2RGLMME      0.199781    -0.050026    -0.118153    -0.103532    -0.112448   
BUCV2TE          0.021484     0.001604    -0.099520     0.022666     0.081219   
BUCV5RGLMME      0.195225    -0.048886    -0.115838    -0.101209    -0.110054   
BUCV5TE          0.021490     0.001541    -0.098841     0.022551     0.080667   
CBE             -0.061033    -0.087015    -0.054172     0.059965     0.084446   
supervised      -0.019663    -0.029024    -0.008747     0.022457     0.028762   
CE               0.034251     0.062144     0.175205     0.011035     0.010035   
unsupervised     0.019475     0.059911     0.135937     0.000420    -0.003599   
CV10RGLMME       0.035197     0.084114    -0.147939    -0.060197    -0.141293   
cvglmm           0.032713     0.074343    -0.133088    -0.054298    -0.126418   
CV10TE          -0.196046     0.119956    -0.000685    -0.060068    -0.013219   
cvte            -0.176754     0.108126    -0.001331    -0.054047    -0.011436   
CV2RGLMME        0.032664     0.073961    -0.132532    -0.054099    -0.125872   
CV2TE           -0.176610     0.108055    -0.001287    -0.054015    -0.011458   
CV5RGLMME        0.031944     0.073916    -0.130745    -0.053491    -0.124909   
CV5TE           -0.176542     0.108020    -0.001243    -0.054016    -0.011505   
DE               0.019651     0.059397     0.135764     0.000828    -0.002949   
DTEM10           0.003316    -0.140106    -0.016668     0.025025     0.055830   
dte              0.001373    -0.093131    -0.011035     0.017100     0.037889   
DTEM2            0.000570    -0.087390    -0.011092     0.016804     0.036138   
DTEM5            0.000516    -0.088429    -0.011249     0.017025     0.036573   
ME01E           -0.010447    -0.000106     0.152292     0.037343    -0.087065   
mee             -0.007088    -0.000697     0.098664     0.025018    -0.055611   
ME10E           -0.008243    -0.002969     0.091773     0.024274    -0.050081   
ME1E            -0.007037    -0.001554     0.100501     0.026642    -0.055647   
MHE              0.019456     0.060241     0.136305     0.000170    -0.004063   
OE               0.016393     0.091523     0.172926    -0.010972    -0.021234   
OHE              0.016410     0.091653     0.173153    -0.010994    -0.021275   
PBTE0001        -0.027983     0.005486     0.066667     0.052915     0.094247   
pbte            -0.019312     0.003015     0.046558     0.037108     0.064726   
PBTE001         -0.018153     0.001319     0.041969     0.034495     0.060527   
PBTE01          -0.017693     0.001887     0.042754     0.035087     0.060590   
RGLMME           0.108382    -0.016628    -0.203275    -0.074025    -0.105861   
glmm             0.051059    -0.004548    -0.070269    -0.032674    -0.047217   
SE               0.018670     0.060108     0.134097    -0.000473    -0.004912   
TE              -0.128071     0.031585    -0.107687     0.006179     0.089850   
te              -0.045196     0.016202    -0.032776    -0.000872     0.027982   
WOEE            -0.017894    -0.026702    -0.008717     0.019683     0.025848   
encoder         -0.003951     0.006695     0.039927     0.010067     0.012664   

              dimension_6  dimension_7  dimension_8  dimension_9  \
BE              -0.106589    -0.093517     0.066265     0.127568   
identifier      -0.103923    -0.091163     0.064598     0.124365   
BUCV10RGLMME     0.128327     0.053959     0.323291     0.090391   
bucvglmm       

## Visualization & Evaluation

In [185]:
# Plot the embedding in an 2D scatter plot with a widget to select the level to plot
def plot_embeddigs(data, levels, subtitle='', dim=2):
    # filter df
    data_filtered = data[data['level'].isin(levels)]
    # plot
    fig = plt.figure(figsize=(11, 8))
    if dim == 2:
        sns.scatterplot(data=data_filtered, x='dimension_1', y='dimension_2', size='degree', sizes=(80, 400), hue='level', palette='Set2')
    elif dim == 3:
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(data_filtered['dimension_1'], data_filtered['dimension_2'], data_filtered['dimension_3'], c=data_filtered['level'], cmap='Set2', s=data_filtered['degree'])
    # Add node labels as annotations to the plot
    for node, (x, y, level, degree) in data_filtered.iterrows():
        plt.text(x, y, node, fontsize=10)
    # Add title to the plot
    plt.title("Poincaré Embedding" + f"{subtitle}", fontweight='bold')
    # Add legend
    plt.legend(labelspacing=1, borderpad=1, loc='upper right', bbox_to_anchor=(1.2, 1))
    # Name axes
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    # Show plot
    plt.tight_layout()
    plt.show()

### Dimensionality reduction t-SNE

In [145]:
# Perform dimensionality reduction using t-SNE (2 dimensions)
tsne = TSNE(n_components=2, random_state=7)
embeddings_2d_tsne = tsne.fit_transform(embeddings_df.values)
embeddings_2d_df_tsne = pd.DataFrame(embeddings_2d_tsne, index=embeddings_df.index, columns=['dimension_1', 'dimension_2'])

In [152]:
# Add hierarchical level and degree as a column to the dataframe where node names are the index
embeddings_2d_df_tsne['level'] = nx.get_node_attributes(G, 'hierarchical_level').values()
embeddings_2d_df_tsne['degree'] = pd.Series(degree_dict)
embeddings_2d_df_tsne

dimension_1  dimension_2  level  degree
node                                                 
BE              -0.035713    -7.154251      0       1
identifier      -0.253479    -7.089592      1       4
BUCV10RGLMME    -0.266439    -7.085462      0       1
bucvglmm        -0.274370    -7.081058      1       4
BUCV10TE        -0.036349    -6.652682      0       1
bucvte           0.445171    -2.680162      1       4
BUCV2RGLMME      0.421230    -2.801278      2       1
BUCV2TE          0.420141    -2.808120      2       1
BUCV5RGLMME      0.417889    -2.816706      2       1
BUCV5TE          0.075423    -3.908264      2       1
CBE             -2.053321    -4.525012      0       1
supervised      -1.915087    -4.550978      1       8
CE              -1.891443    -4.560370      0       1
unsupervised    -1.896829    -4.557077      2       6
CV10RGLMME      -0.744617    -4.978672      0       1
cvglmm           1.536068    -4.857019      1       4
CV10TE           0.421244    -4.789008      0       1
cvte             0.426441    -4.794442      1       4
CV2RGLMME        0.954220    -5.456527      2       1
CV2TE            0.103382    -5.361728      2       1
CV5RGLMME       -0.165339    -4.779289      2       1
CV5TE            1.068315    -4.340006      2       1
DE               0.161373    -7.035540      3       1
DTEM10          -0.154188    -6.577847      0       1
dte             -0.110235    -6.601113      2       4
DTEM2           -0.113956    -6.610617      3       1
DTEM5           -0.644883    -2.921550      3       1
ME01E           -0.560434    -3.084298      0       1
mee             -0.571365    -3.068600      2       4
ME10E           -0.561715    -3.078761      3       1
ME1E            -1.970424    -5.606949      3       1
MHE             -1.839165    -5.551438      3       1
OE              -1.834088    -5.548400      2       1
OHE             -1.844569    -5.556583      2       1
PBTE0001         1.295686    -5.665260      0       1
pbte             0.923444    -5.446465      2       4
PBTE001          0.953731    -5.458287      3       1
PBTE01          -0.306739    -4.727721      3       1
RGLMME          -0.152433    -4.785934      0       1
glmm            -0.160742    -4.786584      2       4
SE               1.487514    -4.178539      3       1
TE               1.100621    -4.328515      0       1
te               1.099287    -4.327594      2       4
WOEE            -0.009653    -2.933204      2       1
encoder         -1.880187    -5.026086      3       2

In [186]:
plot_embeddigs(levels=[0, 1, 2, 3, 4], data=embeddings_2d_df_tsne, subtitle='\n'+'with 2D Dimensionality Reduction (t-SNE)')

Findings:
* The embedding is not very good, but it is possible to see that the nodes are clustered by their hierarchical level.

### Dimensionality reduction PCA

In [82]:
# Perform dimensionality reduction using PCA
pca = PCA(n_components=2, random_state=7)
embeddings_2d_pca = pca.fit_transform(embeddings_df.values)
embeddings_2d_df_pca = pd.DataFrame(embeddings_2d_pca, index=embeddings_df.index, columns=['dimension_1', 'dimension_2'])

In [191]:
# Add hierarchical level and degree as a column to the dataframe where node names are the index
embeddings_2d_df_pca['level'] = nx.get_node_attributes(G, 'hierarchical_level').values()
embeddings_2d_df_pca['degree'] = pd.Series(degree_dict)
embeddings_2d_df_pca

dimension_1  dimension_2  level  degree
node                                                 
BE               0.894882    -0.088386      0       1
identifier       0.871478    -0.086264      1       4
BUCV10RGLMME     0.871263    -0.086033      0       1
bucvglmm         0.870182    -0.085990      1       4
BUCV10TE         0.698040    -0.089902      0       1
bucvte          -0.195911     0.673013      1       4
BUCV2RGLMME     -0.183978     0.611652      2       1
BUCV2TE         -0.183282     0.608311      2       1
BUCV5RGLMME     -0.182811     0.605897      2       1
BUCV5TE         -0.159926     0.303233      2       1
CBE             -0.404382    -0.597065      0       1
supervised      -0.368276    -0.535069      1       8
CE              -0.363480    -0.527974      0       1
unsupervised    -0.364190    -0.528574      2       6
CV10RGLMME      -0.214602    -0.232992      0       1
cvglmm          -0.511633     0.069392      1       4
CV10TE          -0.191329     0.010933      0       1
cvte            -0.179100     0.022204      1       4
CV2RGLMME       -0.119442     0.015615      2       1
CV2TE            0.144240    -0.016074      2       1
CV5RGLMME       -0.081931     0.015616      2       1
CV5TE           -0.076224     0.019518      2       1
DE               0.867800    -0.134300      3       1
DTEM10           0.668250    -0.080852      0       1
dte              0.680584    -0.086795      2       4
DTEM2            0.684634    -0.086834      3       1
DTEM5           -0.162236     0.659600      3       1
ME01E           -0.152302     0.575121      0       1
mee             -0.149898     0.578675      2       4
ME10E           -0.149941     0.575646      3       1
ME1E            -0.279803    -0.648073      3       1
MHE             -0.256796    -0.575145      3       1
OE              -0.257128    -0.572986      2       1
OHE             -0.257851    -0.577533      2       1
PBTE0001        -0.133436     0.016445      0       1
pbte            -0.112442     0.014418      2       4
PBTE001         -0.118169     0.015509      3       1
PBTE01          -0.078235     0.016042      3       1
RGLMME          -0.081505     0.015079      0       1
glmm            -0.075327     0.017610      2       4
SE              -0.075512     0.024723      3       1
TE              -0.075081     0.019817      0       1
te              -0.073922     0.019414      2       4
WOEE            -0.379302     0.824394      2       1
encoder         -0.601969    -0.691034      3       2

In [174]:
plot_embeddigs(levels=[0, 1, 2, 3, 4], data=embeddings_2d_df_pca, subtitle='\n'+'with 2D Dimensionality Reduction (PCA)')

Findings:
* The nodes are obviously somehow clustered, but it is not clear if this directly relates to the level oder degree of the node.